#### Purpose: Retrieve short-term rental json data from Mashvisor API
Source: https://www.mashvisor.com/api-doc/#short-term-rentals

In [1]:
# Import the dependencies.
import json
import numpy
# Import the requests library.
import requests

# Import the API key.
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

# Now do your import
from config import mash_api_key
# from folder.config import weather_api_key

In [2]:
# Adjust the state, city, and page_num for each data pull. Mashviser doesn't allow you to pull the entire
# data in one shot. Initially, we pull the first page, and then identify how many pages to pulls from there
state = "FL"
city = "Tampa"
page_num = 1

# Get Listings
# Lists all active short-term rentals for a specific location: city, zip code, or neighborhood.
filename = "listings_" + city
baseURL = "https://api.mashvisor.com/v1.1/client/airbnb-property/active-listings?state="+ state + "&city=" + city + "&items=20&page=" + str(page_num)

# Get Market Summary
# # This endpoint retrieves a summary and overview for a specific location: city, zip code, or neighborhood.
# filename = "market_summary_" + city
# baseURL = "https://api.mashvisor.com/v1.1/client/airbnb-property/market-summary?state=" + state + "&city=" + city

# # Get Airbnb Occupancy Rates
# # For each Airbnb listing, we calculate its occupancy rate (month per month and annual rates), plus 12-month historical occupancy data.
# filename = "occupancy_rate_" + city
# baseURL = "https://api.mashvisor.com/v1.1/client/airbnb-property/occupancy-rates?state=" + state + "&city=" + city

print(baseURL)

https://api.mashvisor.com/v1.1/client/airbnb-property/active-listings?state=FL&city=Tampa&items=20&page=1


In [3]:
# Add the Authorization header
headers = {'x-api-key': f'{mash_api_key}'}

# Get the list of devices from Nautobot using the requests module and passing in the authorization header defined above
response = requests.get(baseURL, headers=headers)

if response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json"):
    try:
        data = response.json()
    except ValueError:
        print('Error')

In [4]:
data

{'status': 'error', 'code': 401, 'message': 'Api quota has been consumed'}

In [9]:
# Get the total pages for this search
pages = data["content"]["total_pages"]

# Get the properties that are a part of page 1 content
listings = data["content"]["properties"]
 
# Start on page 2 and loop through the number of pages to 
# get properties for each page by sending another JSON API 
# request for that particular page
i = 2
while i < pages:
    baseURL=f"https://api.mashvisor.com/v1.1/client/airbnb-property/active-listings?state={state}&city={city}&items=20&page={str(i)}"
    print(baseURL)
    response = requests.get(baseURL, headers=headers)
    if response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json"):
        try:
            new_data = response.json()
            for listing in new_data["content"]["properties"]:
                listings.append(listing)
        except ValueError:
            print('Error')
    i = i + 1 

# Save data into json file
with open('original_json_data/' + filename + '.json', 'w') as outfile:
    json.dump(listings, outfile)

print("all done")


KeyError: 'content'

In [ ]:
response